

# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [231]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-1f11db047b-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Club explorative search") 

Consider the following exploratory information need:

> investigate the association football clubs and find the main BGPs related to this sport. Compare clubs in countries and championships won. Explore also the participation of the countries in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

wdt:P279## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football club

2. Identify the BGP for association football league

3. Return for each north american country the number of association football clubs

4. Identify the BGP for the UEFA Champions League.

5. Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

6. Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

7. Identify the BGP of the FIFA World Cup

8. Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

9. Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).


## Task 1
Identify the BGP for association football club

In [232]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [233]:
# Print the Object using node Cristiano Ronaldo Q11571
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q11571 ?p ?object .
   ?object sc:name ?name
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'FWA Footballer of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', "PFA Fans' Player of the Year")]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'Liga Portugal')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('name', 'Cristiano')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Commander of the Order of Merit of Portugal')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'Portugal Olympic football team')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'UEFA Euro 2020')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('name', 'dos Santos')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'UEFA Team of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'FIFA World Player of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'FIFA World Player of the Year')]
[('p', 'http:

In [234]:

queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q11571 ?p ?object .
   
   ?object sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1750244'), ('name', 'FWA Footballer of the Year')]
[('object', 'http://www.wikidata.org/entity/Q10726649'), ('name', "PFA Fans' Player of the Year")]
[('object', 'http://www.wikidata.org/entity/Q182994'), ('name', 'Liga Portugal')]
[('object', 'http://www.wikidata.org/entity/Q12899500'), ('name', 'Cristiano')]
[('object', 'http://www.wikidata.org/entity/Q20888805'), ('name', 'Commander of the Order of Merit of Portugal')]
[('object', 'http://www.wikidata.org/entity/Q3590758'), ('name', 'Portugal Olympic football team')]
[('object', 'http://www.wikidata.org/entity/Q373501'), ('name', 'UEFA Euro 2020')]
[('object', 'http://www.wikidata.org/entity/Q20819738'), ('name', 'dos Santos')]
[('object', 'http://www.wikidata.org/entity/Q1146578'), ('name', 'UEFA Team of the Year')]
[('object', 'http://www.wikidata.org/entity/Q182529'), ('name', 'FIFA World Player of the Year')]
[('object', 'http://www.wikidata.org/entity/Q260117'), ('name', "UEF

In [160]:

queryString = """
SELECT DISTINCT ?p ?name 
WHERE {

   wd:Q267245 ?p ?object .
   
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3441'), ('name', 'FIFA country code')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 'ranking')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('name', 'history of topic')]
[('p', 'http://www.wikidata.org/prop/dir

Final query for this task

In [235]:

queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q267245 wdt:P279 ?object .
   
   ?object sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
1


In [236]:

queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q267245 wdt:P31 ?object .
   
   ?object sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
1


In [237]:

queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q6979593 wdt:P279 ?object .
   
   ?object sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('object', 'http://www.wikidata.org/entity/Q1194951'), ('name', 'national sports team')]
2


In [238]:

queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q15944511 wdt:P279 ?object .
   
   ?object sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q28083137'), ('name', 'football team')]
1


In [239]:

queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q15656 wdt:P279 ?object .
   
   ?object sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [243]:
#Try Looking at MANCHESTER UNITED Q18656 Properties
# [('object', 'http://www.wikidata.org/entity/Q18656'), ('name', 'Manchester United F.C.')]




queryString = """
SELECT DISTINCT ?p ?name 
WHERE {

   wd:Q18656 ?p ?object .
   
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1278'), ('name', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1320'), ('name', 'OpenCorporates ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1430'), ('name', 'OpenPlaques subject ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('name', 'official name

In [249]:
# Try Looking at all object of association football P641
# [('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]



queryString = """
SELECT DISTINCT ?sName ?objectName
WHERE {

   ?s wdt:P641 ?object .
   ?s sc:name ?sName.
   
   ?object sc:name ?objectName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'sports in Sweden'), ('objectName', 'cycling')]
[('sName', '2015 Canary Wharf Squash Classic'), ('objectName', 'squash')]
[('sName', 'Canary Wharf Squash Classic'), ('objectName', 'squash')]
[('sName', '2013 Canary Wharf Squash Classic'), ('objectName', 'squash')]
[('sName', '2014 FIM Motocross World Championship season'), ('objectName', 'motocross')]
[('sName', 'beach soccer in Estonia'), ('objectName', 'beach soccer')]
[('sName', 'Rita Keitmann'), ('objectName', 'triathlon')]
[('sName', 'Till Schramm'), ('objectName', 'triathlon')]
[('sName', 'Trevor Wurtele'), ('objectName', 'triathlon')]
[('sName', 'Pien Keulstra'), ('objectName', 'triathlon')]
[('sName', 'Mai Shiina'), ('objectName', 'karate')]
[('sName', 'Keisuke Nemoto'), ('objectName', 'karate')]
[('sName', 'Katsutoshi Shiina'), ('objectName', 'karate')]
[('sName', "karate at the 2006 Asian Games – men's kumite +80 kg"), ('objectName', 'karate')]
[('sName', "karate at the 2006 Asian Games – men's kumite 75 kg

In [250]:
#Try Looking at all objects of association football P641 that are related to CR7

queryString = """
SELECT DISTINCT ?sName ?objectName 
WHERE {

   wd:Q11571 wdt:P641 ?object .
   
   ?object sc:name ?objectName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('objectName', 'association football')]
1


In [251]:
#Looking at association football Q2736

queryString = """
SELECT DISTINCT ?sName ?objectName 
WHERE {

   wd:Q2736 ?p ?object .
   
   ?object sc:name ?objectName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('objectName', 'FIFA')]
[('objectName', 'association football goal')]
[('objectName', 'Category:Association football')]
[('objectName', 'football')]
[('objectName', 'United Kingdom of Great Britain and Ireland')]
[('objectName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('objectName', 'Olympic sport')]
[('objectName', 'team sport')]
[('objectName', 'Armenian Soviet Encyclopedia')]
[('objectName', 'type of sport')]
[('objectName', 'Great Encyclopedia of Cyril and Methodius')]
[('objectName', 'Wikipedia:Vital articles')]
[('objectName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('objectName', 'association football player')]
[('objectName', 'soccer ball')]
[('objectName', 'association football pitch')]
[('objectName', 'football boots')]
[('objectName', 'history of association football')]
[('objectName', 'kit')]
[('objectName', 'WikiProject Football')]
[('objectName', 'spectator sport')]
[('objectName', 'Portal:Association football')]
22


In [252]:
#Looking at association football Q2736

queryString = """
SELECT DISTINCT ?object ?objectName 
WHERE {

   wd:Q2736 ?p ?object .
   
   ?object sc:name ?objectName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q253414'), ('objectName', 'FIFA')]
[('object', 'http://www.wikidata.org/entity/Q571251'), ('objectName', 'association football goal')]
[('object', 'http://www.wikidata.org/entity/Q5611338'), ('objectName', 'Category:Association football')]
[('object', 'http://www.wikidata.org/entity/Q1081491'), ('objectName', 'football')]
[('object', 'http://www.wikidata.org/entity/Q174193'), ('objectName', 'United Kingdom of Great Britain and Ireland')]
[('object', 'http://www.wikidata.org/entity/Q19180675'), ('objectName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('object', 'http://www.wikidata.org/entity/Q212434'), ('objectName', 'Olympic sport')]
[('object', 'http://www.wikidata.org/entity/Q216048'), ('objectName', 'team sport')]
[('object', 'http://www.wikidata.org/entity/Q2657718'), ('objectName', 'Armenian Soviet Encyclopedia')]
[('object', 'http://www.wikidata.org/entity/Q31629'), ('objectName', 'type of sport')]
[('object', 'htt

In [253]:
#Try to look at all the properties related to association football player


queryString = """
SELECT DISTINCT ?p ?pName 
WHERE {

   wd:Q2736 ?p ?object .
   
   ?p sc:name ?pName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3719'), ('pName', 'regulated by')]
[('p', 'http://www.wikidata.org/prop/direct/P10'), ('pName', 'video')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pName', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pName', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pName', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1190'), ('pName', 'Universal Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pName', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pName', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('pName', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalan

In [254]:
#Try to look at the object related to association football player Q937856
# [('object', 'http://www.wikidata.org/entity/Q937857'), ('objectName', 'association football player')]



queryString = """
SELECT DISTINCT ?object ?objectName 
WHERE {

   wd:Q937857 ?p ?object .
   
   ?object sc:name ?objectName.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/P2369'), ('objectName', 'Soccerway player ID')]
[('object', 'http://www.wikidata.org/entity/P3908'), ('objectName', 'Reprezentacija player ID')]
[('object', 'http://www.wikidata.org/entity/Q18536342'), ('objectName', 'competitive player')]
[('object', 'http://www.wikidata.org/entity/P2446'), ('objectName', 'Transfermarkt player ID')]
[('object', 'http://www.wikidata.org/entity/Q7217464'), ('objectName', 'Category:Association football players')]
[('object', 'http://www.wikidata.org/entity/P2823'), ('objectName', 'Royal Belgian Football Association player ID')]
[('object', 'http://www.wikidata.org/entity/Q15944511'), ('objectName', 'association football team')]
[('object', 'http://www.wikidata.org/entity/Q2736'), ('objectName', 'association football')]
[('object', 'http://www.wikidata.org/entity/Q28640'), ('objectName', 'profession')]
[('object', 'http://www.wikidata.org/entity/Q476028'), ('objectName', 'association football club')]
[('

In [255]:
# I have searched the association football club for Q476028


queryString = """
SELECT DISTINCT ?p ?name 
WHERE {

   wd:Q476028 ?p ?object .
   
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Q

In [256]:
# write your final query
# [('object', 'http://www.wikidata.org/entity/Q937857'), ('objectName', 'association football player')]


queryString = """
SELECT DISTINCT ?object ?objectName 
WHERE {

   wd:Q937857 ?p ?object .
   
   ?object sc:name ?objectName
   FILTER REGEX(?objectName, "association football club")
}
LIMIT 45
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q476028'), ('objectName', 'association football club')]
1


In [259]:
# write your final query
# [('object', 'http://www.wikidata.org/entity/Q476028'), ('objectName', 'association football club')]


queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q476028 ?object ?o.
   # get the label
   ?object sc:name ?name.
    
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('object', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('object', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('object', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('object', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('object', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('object', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('object', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('object', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('object', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('object', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('object

## Task 2
Identify the BGP for association football league

In [64]:
# write your queries

queryString = """
SELECT DISTINCT ?p ?name 
WHERE {

   wd:Q15804 ?p ?object .
   
   ?p sc:name ?name
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p',

In [65]:
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {

   wd:Q15804 ?p ?object .
   
   ?object sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('name', 'Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', 'Category:Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('name', 'August')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'FC Internazionale Milano')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'association football competition')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'association football league')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', "men's association football")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'national championship')]
[('p', 'http://www.wikidata.org/prop/direct/P6587'), ('name', 'Italian association football league system')]
[('p', 'http://www.wikidata.org/prop/

Final query for this task

In [49]:
# write your final query
queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q15804 ?p ?object .
   
   ?object sc:name ?name.
   FILTER REGEX(?name, "association football league")
}
LIMIT 45
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
[('object', 'http://www.wikidata.org/entity/Q953425'), ('name', 'Italian association football league system')]
2


## Task 3
Return for each north american country the number of association football clubs

In [72]:
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName 
WHERE {

   ?s ?p wd:Q476028.
   
   ?s sc:name ?sName.
   ?p sc:name ?pName
   
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5611964'), ('sName', 'Template:Infobox football club'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q51481377'), ('sName', "women's association football club"), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q15944511'), ('sName', 'association football team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q28083137'), ('sName', 'football team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q103231176'), ('sName', "men's association football club"), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q51481377'), ('sName', "women's association football club"), ('p', 'http://ww

In [75]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {

   wd:Q476028 ?p ?obj.
   
    ?obj sc:name ?name.
   
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q51481377'), ('name', "women's association football club")]
[('obj', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q628099'), ('name', 'association football manager')]
[('obj', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
[('obj', 'http://www.wikidata.org/entity/Q714581'), ('name', 'Nea Salamis Famagusta FC')]
[('obj', 'http://www.wikidata.org/entity/Q167270'), ('name', 'trademark')]
[('obj', 'http://www.wikidata.org/entity/Q17270000'), ('name', 'football club')]
[('obj', 'http://www.wikidata.org/entity/Q28083137'), ('name', 'football team')]
[('obj', 'http://www.wikidata.org/entity/P5424'), ('name', 'Lega Serie A team ID')]
[('obj', 'http://www.wikidata.org/entity/Q5614332'), ('name', 'Category:Association football clubs')]
[('obj', 'http

In [82]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q476028 ?p ?obj.
   
    ?p sc:name ?name.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Q

In [83]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q1422 ?p ?obj.
   
    ?p sc:name ?name.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('

In [85]:
queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   wd:Q1422 wdt:P17 ?object.
   
    ?object sc:name ?name.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
1


In [86]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q38 ?p ?object.
   
    ?p sc:name ?name.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [88]:
queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   wd:Q38 wdt:P30 ?object.
   
    ?object sc:name ?name.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
1


In [90]:
queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   ?s wdt:P30 ?object.
   
    ?object sc:name ?name.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q60140'), ('name', 'Indian subcontinent')]
[('object', 'http://www.wikidata.org/entity/Q55643'), ('name', 'Oceania')]
[('object', 'http://www.wikidata.org/entity/Q19464773'), ('name', 'Northern America and Mexico')]
[('object', 'http://www.wikidata.org/entity/Q1239'), ('name', 'Indian Ocean')]
[('object', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('object', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
[('object', 'http://www.wikidata.org/entity/Q4412'), ('name', 'West Africa')]
[('object', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('object', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('object', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('object', 'http://www.wikidata.org/entity/Q51'), ('name', 'Antarctica')]
[('object', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('object', 'http://www.wikidata.org/entity/Q5401'),

In [98]:
queryString = """
SELECT DISTINCT ?club ?name
WHERE {

   ?club wdt:P31 wd:Q476028.
   ?club wdt:P17 ?country.
   ?country wdt:P30 wd:Q49.
   
    ?club sc:name ?name.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q7280157'), ('name', "Radford Highlanders men's soccer")]
[('club', 'http://www.wikidata.org/entity/Q15876988'), ('name', 'Grenades FC')]
[('club', 'http://www.wikidata.org/entity/Q3930989'), ('name', 'Real Colorado Foxes')]
[('club', 'http://www.wikidata.org/entity/Q4816106'), ('name', 'Atlanta Magic')]
[('club', 'http://www.wikidata.org/entity/Q4852791'), ('name', 'Baltimore Bohemians')]
[('club', 'http://www.wikidata.org/entity/Q5020506'), ('name', 'California Gold')]
[('club', 'http://www.wikidata.org/entity/Q5087877'), ('name', 'Chattanooga Express')]
[('club', 'http://www.wikidata.org/entity/Q5095562'), ('name', 'Chicago Croatian')]
[('club', 'http://www.wikidata.org/entity/Q5266043'), ('name', 'Waza Flo')]
[('club', 'http://www.wikidata.org/entity/Q6116983'), ('name', 'Jackson Chargers')]
[('club', 'http://www.wikidata.org/entity/Q7013918'), ('name', 'New York Red Bulls U-23')]
[('club', 'http://www.wikidata.org/entity/Q7389397')

Final query for this task

In [ ]:
# write your final query


## Task 4
Identify the BGP for the UEFA Champions League.

In [99]:
# write your queries
#From Previous Result we now look at the properties of Association Football league Q15991303

queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q15991303 ?p ?object.
   ?p sc:name ?name
   
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
9


In [101]:
#we now look at the objects related to Association football league

queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q15991303 ?p ?object.
   ?object sc:name ?name
   
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'sports league level')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'association football team season')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'association football competition')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'sports league')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'organizer')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', 'C

In [109]:
#Filter the data for Champions League

queryString = """
SELECT DISTINCT ?s ?name
WHERE {

   ?s ?p wd:Q15991303.
   ?s sc:name ?name.
   FILTER REGEX (?name, ".*Champions League")
   
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q17632439'), ('name', 'VFF Champions League')]
1


In [110]:
#Filter the data for Champions League

queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   ?s ?p wd:Q15991303.
   ?p sc:name ?name.
   
   
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
9


Final query for this task

In [112]:
#Filter the data for Champions League

queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   ?s wdt:P3450 ?object.
   ?object sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)


Results
[('object', 'http://www.wikidata.org/entity/Q29762'), ('name', 'fencing at the Summer Olympics')]
[('object', 'http://www.wikidata.org/entity/Q3066961'), ('name', 'Galician trainera league')]
[('object', 'http://www.wikidata.org/entity/Q610674'), ('name', 'soccer in the United States')]
[('object', 'http://www.wikidata.org/entity/Q7854'), ('name', 'Argentine Grand Prix')]
[('object', 'http://www.wikidata.org/entity/Q1070262'), ('name', 'European Touring Car Cup')]
[('object', 'http://www.wikidata.org/entity/Q1774675'), ('name', 'División de Honor de Waterpolo')]
[('object', 'http://www.wikidata.org/entity/Q2023337'), ('name', 'División de Honor Juvenil de Fútbol')]
[('object', 'http://www.wikidata.org/entity/Q204348'), ('name', 'Continental Basketball Association')]
[('object', 'http://www.wikidata.org/entity/Q2744659'), ('name', 'Copa de la Reina de Baloncesto')]
[('object', 'http://www.wikidata.org/entity/Q303871'), ('name', 'European Rally Championship')]
[('object', 'http:/

In [133]:
#Filter the data for Champions League

queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   ?s wdt:P3450 ?object.
   ?object sc:name ?name.
   FILTER REGEX(?name, ".*Champions League.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('object', 'http://www.wikidata.org/entity/Q652498'), ('name', "EHF Women's Champions League")]
[('object', 'http://www.wikidata.org/entity/Q648474'), ('name', 'OFC Champions League')]
[('object', 'http://www.wikidata.org/entity/Q300098'), ('name', 'CEV Champions League')]
[('object', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q219261'), ('name', 'CAF Champions League')]
[('object', 'http://www.wikidata.org/entity/Q3240479'), ('name', 'African Handball Champions League')]
[('object', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q941782'), ('name', 'LEN Champions League')]
[('object', 'http://www.wikidata.org/entity/Q22340390'), ('name', 'Basketball Champions League')]
[('object', 'http://www.wikidata.org/e

In [121]:
# write your final query

#Filter the data for UEFA Champions League

queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   ?s wdt:P3450 ?object.
   ?object sc:name ?name.
   FILTER REGEX(?name, "UEFA Champions League")
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
4


## Task 5
Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

In [125]:
# write your queries
# From Previous Result We have UEFA Champions League with Property Q18756
# Check all Champions League winners


queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q18756 ?p ?object.
   ?p sc:name ?name.
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), ('name', 'event interval')]
[('p', 'http:/

In [128]:
# write your queries
# From Previous Result We have UEFA Champions League with Property Q18756
# Check all Champions League winners


queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q18756 wdt:P1346 ?object.
   ?object sc:name ?name.
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'FC Internazionale Milano')]
1


In [136]:
# write your queries
# [('object', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]


queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q13416183 ?p ?object.
   ?p sc:name ?name.
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P393'), ('name', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time

In [260]:
# write your queries
# Check the winner properites of League
# [('object', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
# [('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]


queryString = """
SELECT DISTINCT  ?object ?name
WHERE {

   wd:Q13416183 wdt:P1346 ?object.
   ?object sc:name ?name.
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF')]
1


In [150]:
# write your queries
#CHECK The Instances of 2015-2018 Champions League
# # [('object', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]


queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q13416183 wdt:P31 ?object.
   ?object sc:name ?name.
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
1


In [151]:
# write your queries
#CHECK The Instances of 2015-2018 Champions League
queryString = """
SELECT DISTINCT ?object ?name 
WHERE {

   wd:Q13416183 wdt:P3450 ?object.
   ?object sc:name ?name.
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
1


Final query for this task

In [ ]:
# write your final query


## Task 6
Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

In [261]:
# [('object', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]


# write your queries
#CHECK The properties and objects of serie A at Q15804
queryString = """
SELECT DISTINCT ?p ?pname ?object ?objectname
WHERE {

   wd:Q15804 ?p ?object.
   ?p sc:name ?pname.
   ?object sc:name ?objectname
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('pname', 'league level below'), ('object', 'http://www.wikidata.org/entity/Q194052'), ('objectname', 'Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('object', 'http://www.wikidata.org/entity/Q6807104'), ('objectname', 'Category:Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pname', 'season starts'), ('object', 'http://www.wikidata.org/entity/Q122'), ('objectname', 'August')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('object', 'http://www.wikidata.org/entity/Q2736'), ('objectname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('object', 'http://www.wikidata.org/entity/Q38'), ('objectname', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('object', 'http://www.wikidata.org/entity/Q631'), ('objectname', 'FC Internazionale Milano')]
[('p', 

In [185]:

# [('object', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]

# write your queries
#CHECK The properties and objects of serie A at Q15804
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE {

   ?s ?p wd:Q15804.
   ?p sc:name ?pname.
   ?s sc:name ?sname
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q45901'), ('sname', 'Thierry Henry'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league')]
[('s', 'http://www.wikidata.org/entity/Q223724'), ('sname', '2009–10 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q240669'), ('sname', '2011–12 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q276204'), ('sname', '2010–11 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q381013'), ('sname', '2007–08 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q643978'), ('sname', '2005–06 Serie A'), ('p', 'http://www.wikid

In [186]:


# write your queries
#CHECK The properties and objects of serie A at Q15804
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE {

   ?s ?p wd:Q15804.
   ?p sc:name ?pname.
   ?s sc:name ?sname
   FILTER REGEX(?pname, 'top')
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6807104'), ('sname', 'Category:Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q7195347'), ('sname', 'Category:Serie A seasons'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q10956030'), ('sname', 'Template:Serie A seasons'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q7119823'), ('sname', 'Category:Serie A players'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]
4


In [187]:


# write your queries
#CHECK The properties and objects of serie A at Q15804
queryString = """
SELECT DISTINCT ?p ?pname ?object ?objectName
WHERE {

   wd:Q15804 ?p ?object.
   ?p sc:name ?pname.
   ?object sc:name ?objectName
   FILTER REGEX (?pname, 'top')
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('object', 'http://www.wikidata.org/entity/Q6807104'), ('objectName', 'Category:Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template"), ('object', 'http://www.wikidata.org/entity/Q10956030'), ('objectName', 'Template:Serie A seasons')]
2


In [191]:


# write your queries
#Get some insights from Italian association Football league at Q953425
queryString = """
SELECT DISTINCT ?p ?pname ?object ?objectName
WHERE {

   wd:Q953425 ?p ?object.
   ?p sc:name ?pname.
   ?object sc:name ?objectName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('object', 'http://www.wikidata.org/entity/Q2736'), ('objectName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('object', 'http://www.wikidata.org/entity/Q2736'), ('objectName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('object', 'http://www.wikidata.org/entity/Q38'), ('objectName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('object', 'http://www.wikidata.org/entity/Q1756477'), ('objectName', 'league system')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('object', 'http://www.wikidata.org/entity/Q29791211'), ('objectName', 'sport in a geographic region')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('object', 'http://www.wikidata.org/entity/Q8557731'), ('objectName', 'Categor

In [198]:


# write your queries
#Get some insights from Italian association Football league at Q3270632
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {

   ?s ?p wd:Q953425.
   ?p sc:name ?pName.
   ?s sc:name ?sName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q645148'), ('sName', 'Eccellenza'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pName', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q194052'), ('sName', 'Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pName', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q15804'), ('sName', 'Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pName', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q607965'), ('sName', 'Serie C'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pName', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q677397'), ('sName', 'Lega Pro Prima Divisione'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pName', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q1141778'), ('sName', 'Serie D'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pName', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q8557731'), ('sName',

In [201]:


# write your queries
# Look at the National Championship Q3270632
# [('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('object', 'http://www.wikidata.org/entity/Q3270632'), ('objectname', 'national championship')]
queryString = """
SELECT DISTINCT ?p ?pName ?object ?objectName
WHERE {

   wd:Q3270632 ?p ?object.
   ?p sc:name ?pName.
   ?object sc:name ?objectName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('object', 'http://www.wikidata.org/entity/Q1079023'), ('objectName', 'championship')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('object', 'http://www.wikidata.org/entity/Q7481805'), ('objectName', 'Category:National championships')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('object', 'http://www.wikidata.org/entity/Q14358455'), ('objectName', 'Template:Infobox national championship')]
3


In [203]:


# write your queries
# try to EXPLORE ASSOCIATION FOOTBALL LEAGUE Q15991303
queryString = """
SELECT DISTINCT ?p ?pName ?object ?objectName
WHERE {

   wd:Q15991303 ?p ?object.
   ?p sc:name ?pName.
   ?object sc:name ?objectName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('object', 'http://www.wikidata.org/entity/P2499'), ('objectName', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('object', 'http://www.wikidata.org/entity/P2500'), ('objectName', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('object', 'http://www.wikidata.org/entity/P3983'), ('objectName', 'sports league level')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('object', 'http://www.wikidata.org/entity/Q26887310'), ('objectName', 'association football team season')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('object', 'http://www.wikidata.org/entity/P17'), ('objectName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('object', 'ht

In [205]:
# LOOK AT SPORT LEAGUE LEVEL P3983
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {

   ?s ?p wd:P3983.
   ?s sc:name ?sName.
   ?p sc:name ?pName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q623109'), ('sName', 'sports league'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q15991303'), ('sName', 'association football league'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type')]
[('s', 'http://www.wikidata.org/entity/Q106314805'), ('sName', 'volleyball league'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type')]
[('s', 'http://www.wikidata.org/entity/Q75179210'), ('sName', 'handball league'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type')]
[('s', 'http://www.wikidata.org/entity/P2500'), ('sName', 'league level below'), ('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also')]
[('s', 'http://www.wikidata.org/entity/P2499'), ('sName', 'league level above'), ('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName

In [206]:
# LOOK AT SPORT LEAGUE LEVEL P3983
queryString = """
SELECT DISTINCT ?p ?pName ?object ?objectName
WHERE {

   wd:P3983 ?p ?object .
   ?object sc:name ?objectName.
   ?p sc:name ?pName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('object', 'http://www.wikidata.org/entity/Q28106586'), ('objectName', 'Wikidata property related to sport events')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('object', 'http://www.wikidata.org/entity/P2500'), ('objectName', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('object', 'http://www.wikidata.org/entity/P2499'), ('objectName', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('object', 'http://www.wikidata.org/entity/P6587'), ('objectName', 'league system')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('object', 'http://www.wikidata.org/entity/Q623109'), ('objectName', 'sports league')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('object', 'http://www.wiki

In [207]:
# LOOK AT SPORT LEAGUE LEVEL P3983
queryString = """
SELECT DISTINCT ?p ?pName ?object ?objectName
WHERE {

   wd:P3983 ?p ?object .
   ?object sc:name ?objectName.
   ?p sc:name ?pName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('object', 'http://www.wikidata.org/entity/Q28106586'), ('objectName', 'Wikidata property related to sport events')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('object', 'http://www.wikidata.org/entity/P2500'), ('objectName', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('object', 'http://www.wikidata.org/entity/P2499'), ('objectName', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('object', 'http://www.wikidata.org/entity/P6587'), ('objectName', 'league system')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('object', 'http://www.wikidata.org/entity/Q623109'), ('objectName', 'sports league')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('object', 'http://www.wiki

In [262]:


# write your queries
# EXPLORE ASSOCIATION FOOTBALL COMPETITION Q1478437
# [('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('object', 'http://www.wikidata.org/entity/Q1478437'), ('objectName', 'association football competition')]
queryString = """
SELECT DISTINCT ?p ?pName ?object ?objectName
WHERE {

   wd:Q1478437 ?p ?object.
   ?p sc:name ?pName.
   ?object sc:name ?objectName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('object', 'http://www.wikidata.org/entity/Q16510064'), ('objectName', 'sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('object', 'http://www.wikidata.org/entity/Q1079023'), ('objectName', 'championship')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('object', 'http://www.wikidata.org/entity/Q2736'), ('objectName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('object', 'http://www.wikidata.org/entity/Q500834'), ('objectName', 'tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('object', 'http://www.wikidata.org/entity/Q7015591'), ('objectName', 'Category:Association football competitions')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('object', 'http://www.wikidata.org/ent

In [263]:


# write your queries
# Explore national Championship Q3270632
# [('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('object', 'http://www.wikidata.org/entity/Q3270632'), ('objectname', 'national championship')]
queryString = """
SELECT DISTINCT ?p ?pName ?object ?objectName
WHERE {

   wd:Q3270632 ?p ?object.
   ?p sc:name ?pName.
   ?object sc:name ?objectName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('object', 'http://www.wikidata.org/entity/Q1079023'), ('objectName', 'championship')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('object', 'http://www.wikidata.org/entity/Q7481805'), ('objectName', 'Category:National championships')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('object', 'http://www.wikidata.org/entity/Q14358455'), ('objectName', 'Template:Infobox national championship')]
3


In [ ]:
# write your final query

## Task 7
Identify the BGP of the FIFA World Cup

In [212]:
# write your queries
# Try to look all the properties of FIFIA World Cup Q170645
queryString = """
SELECT DISTINCT ?p ?pName ?object ?objectName
WHERE {

   wd:Q170645 ?p ?object .
   ?p sc:name ?pName.
   ?object sc:name ?objectName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('pName', 'final event'), ('object', 'http://www.wikidata.org/entity/Q31043671'), ('objectName', '2018 FIFA World Cup Final')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('object', 'http://www.wikidata.org/entity/Q253414'), ('objectName', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('object', 'http://www.wikidata.org/entity/Q202163'), ('objectName', 'Luzhniki Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('object', 'http://www.wikidata.org/entity/Q159'), ('objectName', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('object', 'http://www.wikidata.org/entity/Q27020041'), ('objectName', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('object', 'http://www.wikidata.org/entity/Q2736'), ('objectName', 'association football')]
[('p', 'http:

Final query for this task

In [213]:
# write your final query
# write your queries
# Try to LOOK at P3450 properties of sport seasion of league of FIFIA World Cup Q170645
queryString = """
SELECT DISTINCT ?p ?pName ?object ?objectName
WHERE {

   wd:Q170645 wdt:P3450 ?object .
   ?object sc:name ?objectName
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q19317'), ('objectName', 'FIFA World Cup')]
1


In [264]:
# write your final query
# write your queries
# Retrived BGP data for FIFA WORLD CUP USING Q19317
# [('object', 'http://www.wikidata.org/entity/Q19317'), ('objectName', 'FIFA World Cup')]


queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q19317 ?object ?o.
   # get the label
   ?object sc:name ?name.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/prop/direct/P7441'), ('name', 'WorldFootball.net competition ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('object', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('object', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('object', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('object', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('object', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('object', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Fac

## Task 8
Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query

## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query